# (WIP) Bond Pricing And Accrual Calculation

This notebook will run through the following business use cases :
* Pricing a Bond using the built-in discounting LUSID model fed with a user supplied OIS yield curve.
* Calculating the accrued interest between coupon dates based on user defined Bond instrument parameters such as the day count convention.
* Adding a transaction in a user defined Bond Instrument to a portfolio and subsequently valuing our portfolio.
* Overriding the calculated accrued interest with user provided value and feeding it in to our Bond valuation.
* <REMOVE - awaiting change. right now feeding in face value makes no real sense> Retrieving Bond PV that has been provided to LUSID (e.g. from a market price source).

In doing so we'll cover the following LUSID concepts :
* Defining a LUSID internal representation of a Bond instrument based on user provided parameters.
* Using the StructureMarketData store to hold your OIS yield curve data in way that enables it to be discovered during the Bond valuation process.
* Configuring recipes to run built in LUSID Bond valuation models that make use of the structured data (OIS Yield Curve) you provided.
* Using aggregation requests to return the accrued interest as well as the PV of the Bond based on our instrument definition.
* Using the StructuredResultData store to override the accrued interest calculation and instead use supplied static values.
* Updating our recipes to make use of the StructuredResultData entries in your valuations as oppose to calculating.
* <REMOVE - as above >Upserting Bond market prices as quotes and configuring recipes that value bonds by simply using the quotes.

For this notebook example we'll work with an example Gilt 1.5% 47s:
* Coupon Rate : 1.5%
* Maturity Date: 22 Jul 2047
* Issue Date: 21 Sep 2016
* Coupon Dates : 22 Jan, 22 Jun
* Face Value : £1

## Setup LUSID

In [1]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)

LUSID Environment Initialised
LUSID SDK Version:  0.6.4695.0


In [127]:
# Setup the scope we'll use for this notebook
scope = "test-bond-pricing-nb"

# Utility function to help display aggregation results clearly
def aggregation_result_to_dataframe(aggregation_results):
    return pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])


## Define our Gilt as a Bond Instrument in LUSID

Let's start by defining our Bond instrument in LUSID via the BondInstrument class in the models of the SDKs. Take a look
in the models package for other instruments currently supported (or see the [BondInstrument Specification](https://www.lusid.com/api/swagger/index.html#model-BondInstrument)).

Start by initialising the basic bond parameters for our Gilt:

In [13]:
coupon_rate = 0.015
start_date = datetime(2016, 9, 21, tzinfo=pytz.utc)
maturity_date = datetime(2047, 7, 22, tzinfo=pytz.utc)
dom_ccy = "GBP"
face_value = 1

# we will experiment with effective_at data further in notebook when working with accruals between coupon payments.
trade_date = datetime(2020, 6, 22, tzinfo=pytz.utc)
effective_at = datetime(2020, 6, 23, tzinfo=pytz.utc)

Let's now move onto describing the conventions our bond instrument follows. Specifically we'll set up the behaviour of
our cash flows date schedule which includes setting the day count convention for calculating accrued interest and
handling cash flows landing on non business days.

All this is behaviour is described  in a FlowConventions object:

In [4]:
gilt_flow_conventions = models.FlowConventions(
        # coupon payment currency
        currency="GBP",
        # semi-annual coupon payments
        payment_frequency=models.Tenor(value=6, unit="M"),
        # using an Actual/365 day count convention (other options : Act360, ActAct, ...
        day_count_convention="Act365",
        # modified following rolling convention (other options : ModifiedPrevious, NoAdjustment, EndOfMonth,...)
        roll_convention="ModifiedFollowing",
        # no holiday calendar supplied
        holiday_calendars=[]
    )

For details on supported tenors, day count and roll conventions see the [BondInstrument Specification](https://www.lusid.com/api/swagger/index.html#model-BondInstrument).

We now have all we need to describe our Gilt as a LUSID instrument:

In [14]:
def create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value, flow_conventions):
    instrument = models.BondInstrument(
        start_date=start_date.isoformat(),
        maturity_date=maturity_date.isoformat(),
        dom_ccy=dom_ccy,
        coupon_rate=coupon_rate,
        principal=face_value,
        flow_conventions=flow_conventions,
        instrument_type="Bond"
    )
    return instrument

bond_instrument_definition = create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value, gilt_flow_conventions)
print(bond_instrument_definition)

{'coupon_rate': 0.015,
 'dom_ccy': 'GBP',
 'flow_conventions': {'currency': 'GBP',
                      'day_count_convention': 'Act365',
                      'holiday_calendars': [],
                      'payment_frequency': {'unit': 'M', 'value': 6},
                      'roll_convention': 'ModifiedFollowing'},
 'identifiers': None,
 'instrument_type': 'Bond',
 'maturity_date': '2047-07-22T00:00:00+00:00',
 'principal': 1,
 'start_date': '2016-09-21T00:00:00+00:00'}


Let's use our Bond instrument definition to create an instance of our Gilt 1.5% 47 and upsert it as an instrument into LUSID.

In [15]:
def upsert_bond_instrument(instrument_id, instrument_name, instrument_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        name=instrument_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        definition=instrument_definition
    )}
    # Using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition.
    return instruments_api.upsert_lusid_instruments(bond_instrument_request)

instrument_upsert_response = upsert_bond_instrument("gilt2047s", "gilt 1.5% 47s", bond_instrument_definition)
# retrieve the instrument id of our gilt to be used later when adding market quotes.
gilt_2047_LUID = instrument_upsert_response.values['gilt2047s'].lusid_instrument_id


{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0RH35Q600E:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'gilt2047s': {'href': 'https://khalid-local-dev.lusid.com/api/api/instruments/LusidInstrumentId/LUID_I1RHNWII',
                          'identifiers': {'ClientInternal': 'gilt2047s',
                                          'LusidInstrumentId': 'LUID_I1RHNWII'},
                          'instrument_definition': {'content': '{\n'
                                                               '  "startDate": '
                        

## Defining the Bond valuation

Now that we have our Bond instrument defined and upserted into LUSID we can move onto preparing to execute aggregations
in LUSID to value our Bond holdings.

Aggregations are configured in LUSID through the use of [Recipes]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used") which allow you to configure behaviour. Examples of the type
of configurations include sources market data for specific asset classes, which pricing models to use and where to locate static values that may
be used in calculations.

We'll start with a recipe that simply informs the aggregation engine of which model to use price our Bond. In this notebook
we'll use LUSID's built-in "Discounting" model that prices our bond using an OIS yield curve. But before we run our valuation
we need to supply our yield curve.


### Structured Market Data

[Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr") expands on the simple uploading of market
quotes by allowing you to supply more complex market data into LUSID in a format it can understand.

In the case of our gilt we would like to price it using an OIS yield curve. The source file is in the 'data/GBPOIS50.json' and
consists of a set of discount factors across the maturities of the OIS term structure. While we've hardcoded the rates
in this example LUSID also supports supplying instrument identifiers.#TODO checkthis?

Let's now load in the yield curves:

In [117]:
# TODO expand as to why this different from other scope
market_data_scope = "FinbourneMarketData"
#
market_supplier = 'Lusid'

def upsert_ois_yield_curve(ois_curve_json, scope, effective_at, market_asset, ccy):

    ## TODO clean up and elaborate as this currently too messy
    ## a lot going on here we need to describe

    # create request
    structuredDoc = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ois_curve_json)
    structuredId = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="MyDemoData", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)
    smdRequest = models.UpsertStructuredMarketDataRequest(market_data_id=structuredId,
                                                          market_data=structuredDoc)

    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={ccy : smdRequest}
    )
    print(response)

def load_ois_curve_json():
    with open('data/GBPOIS50.json', "r") as myfile:
        return myfile.read()

ois_curve_json = load_ois_curve_json()
upsert_ois_yield_curve(ois_curve_json, market_data_scope, effective_at, "GBP/GBPOIS", "GBP")
upsert_ois_yield_curve(ois_curve_json, market_data_scope, effective_at, "GBP/6M", "GBP")

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UJ4J3FI:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'GBP': datetime.datetime(2020, 6, 29, 14, 19, 50, 509352, tzinfo=tzutc())}}
{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UJ4J3FI:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'GBP': datetime.datetime(2020, 6, 29, 14, 19, 50, 810147, tzinfo=tzutc())}}


### What Bond Pricing Model To Use?
To value our Gilt we need to tell LUSID how to answer the following questions - What Bond pricing model to use? And where
should LUSID source the market data to required to properly execute the model? To do so we define a Recipe that has two
main constituents, the PricingContext and the MarketContext.

We start with the PricingContext which is used to select the pricing model and add any additional parameters that configure the
model behaviour. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-PricingContext") for more details.

In [8]:
#TODO explain more of these arguments

def create_discounting_bond_pricing_context():
    return models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True,
            produce_separate_result_for_linear_otc_legs=False,
            allow_partially_successful_evaluation=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ]
    )

pricing_context = create_discounting_bond_pricing_context()

### Where should Market Data be sourced?

The MarketContext is how we inform LUSID where to retrieve market data for a given aggregation. In our case we need to
tell LUSID where our OIS yield curve was stored (the market data scope and supplier). See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
for more details.

In [118]:
def create_market_context():
    return models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our Bond Pricing Recipe

With our PricingContext and MarketContext defined we're now ready to configure our Bond valuation recipe:

In [138]:
def create_discount_bond_pricing_recipe(scope, recipe_name, market_context, pricing_context):

    discount_bond_pricing_config_recpie = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_name,
        description="Price bond using discounting model",
        market=market_context,
        pricing=pricing_context
    )


    #upsert the recipe
    response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=discount_bond_pricing_config_recpie)
    )

    print(response)

    return discount_bond_pricing_config_recpie

discounting_bond_recipe_name = "test-discounting-bond"
discount_bond_pricing_config_recpie = create_discount_bond_pricing_recipe(scope, discounting_bond_recipe_name, market_context, pricing_context)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UJ4J4RL:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2020, 6, 29, 15, 44, 44, 716301, tzinfo=tzutc())}


## Pricing our Bond

Let's quickly summarise our current state:
 * We've defined a Gilt 1.5% 47s bond (including defining it's date conventions).
 * We've loaded the OIS yield curve into the Structure Market Data store.
 * We've setup our Recipe configuring of how we would like to price our bond and where to source our required market data.

At this point we hold no position in the Bond in our portfolio but would simply like to value it using our internal Bond
pricing model as we defined in the recipe.


In [ ]:
# create aggregation request

# def create_aggregation_request(inlineCfgRecipe, effective_date, include_cashflows=True):
#
#     # fill in the common bits of an aggregation request
#     aggregateSpecList = [
#         models.AggregateSpec(key='Analytic/default/ValuationDate', op='Value'),
#         models.AggregateSpec(key='Holding/default/PV', op='Value'),
#         models.AggregateSpec(key='Holding/default/Accrual', op='Value')
#     ]
#     if include_cashflows: aggregateSpecList.append(
#         models.AggregateSpec(key='Analytic/default/HoldingCashflows', op='Value'))
#
#     aggregationRequestResource = models.AggregationRequest(
#         inline_recipe=inlineCfgRecipe,
#         effective_at=effective_date.isoformat(),
#         metrics=aggregateSpecList
#     )
#     return aggregationRequestResource
#
# aggregationRequestResource = create_aggregation_request(
#     cfgRecipe, effectiveAt, include_cashflows=False)
#
# weightedInstrument_gilt = models.WeightedInstrument(quantity=1, holding_identifier="myholding_gilt",
#                                                     instrument=gilt_2047s)
#
# inlineRequestGilt = models.InlineAggregationRequest(
#     request=aggregationRequestResource, instruments=[weightedInstrument_gilt]
# )
#
# # Call LUSID to perform the aggregation
# response = api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
#     market_data_scope, inline_request=inlineRequestGilt
# )

In [128]:
def run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at):
    ## TODO why we need weighted insturments
    weighted_instrument_gilt = models.WeightedInstrument(quantity=1, instrument=bond_instrument_definition, holding_identifier="myholding_gilt")

    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=discount_bond_pricing_config_recpie,
        metrics=[
            # TODO called holding even if we don't have one. Dummy it with weigted instrument. Speak with Riz
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    ## TODO why we include inline
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[weighted_instrument_gilt]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)



result = run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at)
bond_pv = result.data[0]['Holding/default/PV']
#print(f"Bond PV (as at {effective_at}) : {bond_pv}")
print(aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv]
]))

      Name              Effective At     Value
0  Bond PV 2020-06-23 00:00:00+00:00  1.386649


We've now priced our Gilt using the LUSID internal Bond "Discounting" model. As our recipe is already setup
we can seamlessly reprice our bond for the following day:

In [129]:
effective_at_t_plus_one = effective_at + timedelta(days=1)
aggregation_result = run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at_t_plus_one)
bond_pv_at_t_plus_one = aggregation_result.data[0]['Holding/default/PV']
#print(f"Bonds PV (as at {effective_at_plus_one}) : {bond_pv}")

print(aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv],
    ['Bond PV', effective_at_t_plus_one, bond_pv_at_t_plus_one]
]))

      Name              Effective At     Value
0  Bond PV 2020-06-23 00:00:00+00:00  1.386649
1  Bond PV 2020-06-24 00:00:00+00:00  1.386690


## Accrued Interest

The above example shows our Bond price changing from one day to the next using the Discounting model. This can be explained by the accrued
interest on the Gilt. Recall that when we defined our gilt we set FlowConventions that contain all the information needed to calculate
accrued interest between coupon dates.

Note that the The 'Holding/default/PV' aggregation firstly generates a clean price using the "Discounting" model before
calculating and adding the accrued interest. To get back the accrued interest taht was calculated we simply need to update
our aggregation request to include 'Holding/default/Accrual':

In [141]:
def run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at):
    ## TODO why we need weighted insturments
    weighted_instrument_gilt = models.WeightedInstrument(quantity=1, instrument=bond_instrument_definition, holding_identifier="myholding_gilt")

    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=discount_bond_pricing_config_recpie,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
            # Ensure the calculated accrual is returned.
            models.AggregateSpec(key='Holding/default/Accrual', op='Value')
        ]
    )

    ## TODO why we include inline
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[weighted_instrument_gilt]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)


# bond pv and accrued interest at effective date
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at)
bond_pv = aggregation_result.data[0]['Holding/default/PV']
accrued_interest = aggregation_result.data[0]['Holding/default/Accrual']

# bond pv and accrued interest day after
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at_t_plus_one)
bond_pv_t_plus_one = aggregation_result.data[0]['Holding/default/PV']
accrued_interest_t_plus_one = aggregation_result.data[0]['Holding/default/Accrual']

print(aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv],
    ['Accrued Interest', effective_at, accrued_interest],
    ['Bond PV', effective_at_t_plus_one, bond_pv_t_plus_one],
    ['Accrued Interest', effective_at_t_plus_one, accrued_interest_t_plus_one],
    ['Bond PV Dtd', effective_at_t_plus_one, bond_pv_t_plus_one-bond_pv],
    ['Accrued Interest Dtd', effective_at_t_plus_one, accrued_interest_t_plus_one-accrued_interest]
]))

                   Name              Effective At     Value
0               Bond PV 2020-06-23 00:00:00+00:00  1.386649
1      Accrued Interest 2020-06-23 00:00:00+00:00  0.006288
2               Bond PV 2020-06-24 00:00:00+00:00  1.386690
3      Accrued Interest 2020-06-24 00:00:00+00:00  0.006329
4           Bond PV Dtd 2020-06-24 00:00:00+00:00  0.000041
5  Accrued Interest Dtd 2020-06-24 00:00:00+00:00  0.000041


## Adding a Bond Position to our Portfolio

Now that we've covered how to define and price a bond instrument let's move onto adding a bond position to our portfolio.
We'll then use the same recipe we defined to price the bond but this time use it to value our entire bond position within
our portfolio. So we're using the same Recipe (i.e same Bond pricing model and data sources) but different aggregation parameters

Brief summary of what we'll aim to do:
* Create a portfolio to hold our Bond position.
* Create a buy transaction on our holding. We'll also make use of the accrual calculation we just covered to help us
setup a realistic transaction price and consideration for our test case.
* Run an aggregation to calculate the value of our portfolio with the accrued interest. We'll also review how this aggregation
on our portfolio differs to the the previous inline aggregation.

But first to begin with let's setup our test portfolio:

In [134]:
portfolio = "simple-bond-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    print(result)

create_portfolio(scope, portfolio, portfolio, "GBP")

{'portfolios': {'errors': [], 'success': [{'base_currency': 'GBP',
 'created': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=tzutc()),
 'description': None,
 'display_name': 'simple-bond-portfolio-01',
 'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/simple-bond-portfolio-01?asAt=2020-06-29T15%3A33%3A42.5770640%2B00%3A00',
 'id': {'code': 'simple-bond-portfolio-01', 'scope': 'test-bond-pricing-nb'},
 'is_derived': False,
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/simple-bond-portfolio-01/properties?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2020-06-29T15%3A33%3A42.5770640%2B00%3A00',
            'method': 'GET',
            'relation': 'Properties'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/simple-bond-portfolio-01/commands?toAsAt=2020-06-29T15%3A33%3A42.5770640%2B00

### Setting up our Transaction

We now need to create and upsert our buy transaction. Earlier we priced the Present Value of our Bond (approx 1.38) which
included accrued interest of around 0.00062. For our example transaction let's set a market price of 1.37 and we're looking
for a position size with a notional of 75,000,000. We'll use the accrued interest calculated by LUSID to come up with a dirty price
and consideration for this particular example:

In [159]:
# retrieve accrued interest
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at)
bond_unit_pv = aggregation_result.data[0]['Holding/default/PV']
accrued_unit_interest = aggregation_result.data[0]['Holding/default/Accrual']

# setup our transaction
notional = 75000000
clean_price = 1.37
dirty_price = clean_price + accrued_unit_interest
consideration = notional * dirty_price

def upsert_buy_transaction(scope, txn_id, instrument_id, trade_date, effective_at, portfolio, clean_price, dirty_price):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type="Buy",
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=effective_at.isoformat(),
        units=notional,
        transaction_price=models.TransactionPrice(price=clean_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency="GBP"),
        exchange_rate=1,
        transaction_currency="GBP"
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])

upsert_buy_transaction(scope, "GiltTXN001", "gilt2047s", trade_date, effective_at, portfolio, clean_price, dirty_price)


## Valuing our Portfolio

We're now ready to run our aggregation and value our Bond portfolio. It's important to note that this is no longer an
inline aggregation. This means the aggregation we're now running isn't only against the bond instrument definition, but instead
is being run against a portfolio within a specific scope. For this reason we no longer require the use of a weighted instrument
or the need to generate a specific inline aggregation request.

Our request is simpler. We just need to provide our recipe, the portfolio and the scope:

In [142]:
def run_bond_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        recipe_id=models.ResourceId(scope=scope, code=recipe),
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
            models.AggregateSpec(key='Holding/default/Accrual', op='Value')
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)



results = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discounting_bond_recipe_name, effective_at)
bond_pv = results.data[0]['Holding/default/PV']
#print(f"Bonds Holdings (as at {effective_at}) : {bond_holdings}")
accrued_interest = results.data[0]['Holding/default/Accrual']
#print(f"Accrued Interest (as at {effective_at}) : {accrued_interest}")

print(aggregation_result_to_dataframe([
    ['Portfolio PV', effective_at, bond_pv],
    ['Portfolio Accrued Interest', effective_at, accrued_interest]
]))

                         Name              Effective At         Value
0                Portfolio PV 2020-06-23 00:00:00+00:00  1.039986e+08
1  Portfolio Accrued Interest 2020-06-23 00:00:00+00:00  4.715753e+05


As in our previous example let's move forward a day to view the change in our Portfolio PV and Accrued Interest:

In [143]:
effective_at_plus_one = effective_at + timedelta(days=1)
results = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discounting_bond_recipe_name, effective_at_plus_one)
bond_pv_t_plus_one = results.data[0]['Holding/default/PV']
accrued_interest_t_plus_one = results.data[0]['Holding/default/Accrual']
#print(f"Bonds Holdings (as at {effective_at}) : {bond_holdings}")
#print(f"Bonds Holdings (as at {effective_at_plus_one}) : {bond_holdings_t_1} (dtd: {bond_holdings_t_1-bond_holdings})")
#print(f"Accrued Interest (as at {effective_at}) : {accrued_interest}")
#print(f"Accrued Interest (as at {effective_at_plus_one}) : {accrued_interest_t_1} (dtd: {accrued_interest_t_1-accrued_interest})")


print(aggregation_result_to_dataframe([
    ['Portfolio PV', effective_at, bond_pv],
    ['Portfolio Accrued Interest', effective_at, accrued_interest],
    ['Portfolio PV', effective_at_t_plus_one, bond_pv_t_plus_one],
    ['Portfolio Accrued Interest', effective_at_t_plus_one, accrued_interest_t_plus_one],
    ['Portfolio PV Dtd', effective_at_t_plus_one, bond_pv_t_plus_one - bond_pv],
    ['Portfolio Accrued Interest Dtd', effective_at_t_plus_one, accrued_interest_t_plus_one - accrued_interest]
]))

                             Name              Effective At         Value
0                    Portfolio PV 2020-06-23 00:00:00+00:00  1.039986e+08
1      Portfolio Accrued Interest 2020-06-23 00:00:00+00:00  4.715753e+05
2                    Portfolio PV 2020-06-24 00:00:00+00:00  1.040017e+08
3      Portfolio Accrued Interest 2020-06-24 00:00:00+00:00  4.746575e+05
4                Portfolio PV Dtd 2020-06-24 00:00:00+00:00  3.082192e+03
5  Portfolio Accrued Interest Dtd 2020-06-24 00:00:00+00:00  3.082192e+03


## Accrual Overrides

Up until now we have relied on LUSID to run it's internal bond pricing model as well as calculate accruals based on the
conventions we defined in the Gilt instrument definition. However you may need to use a different value for accrued interest - one
that has been calculated externally for example. To do so we need to address two concerns. How do we load a one off accrual
into LUSID? And how do we ensure LUSID uses that accrual during aggregation as oppose to reverting to calculating it
on the fly as in the previous examples.

### Structured Result Store

The [Structured Result Store]("https://support.finbourne.com/how-do-i-store-and-retrieve-structured-market-data-documents") is
a location to store non quote data that may nevertheless be used in an aggregation. Examples include YTD performance on
an Index, sensitivities of a Swap, or in our case the accrued interest.

Just as with we did with our OIS yield curves in the Structured Market Data store we need to ensure that the Results we
upsert can be resolved by LUSID during an aggregation. To do so all entries into the Result store must be defined with a
corresponding key that uniquely identifies what the value relates to.

In [145]:
result_data_scope = "Finbourne-Examples"
accrual_result_id = "GiltAccrual"

def upsert_structured_result_data_overrides(effective_at, accrual_result_id, instrument_id):
    accrual_document = "LusidInstrumentId,Accrual" + "\r\n" + f"{instrument_id},0.069109979"

    accrual_result = models.StructuredResultData(
        # TODO change to JSON
        document_format="CSV",
        version="1.0.0",
        name="IRS accrual",
        document=accrual_document
    )

    # create identifier
    accrual_result_id = models.StructuredResultDataId(
        source="Client",
        code=accrual_result_id,
        effective_at=effective_at.isoformat(),
        result_type="UnitResult/Analytic"
    )

    # create structured request
    structured_request = models.UpsertStructuredResultDataRequest(
        id=accrual_result_id,
        data=accrual_result
    )

    # upsert
    response = structured_result_data_api.upsert_structured_result_data(
        scope=result_data_scope,
        request_body={"AccrualOR1": structured_request}
    )

    print(response)

upsert_structured_result_data_overrides(effective_at, accrual_result_id, gilt_2047_LUID)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UMP53CU:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'AccrualOR1': datetime.datetime(2020, 6, 29, 15, 57, 36, 622966, tzinfo=tzutc())}}


Now that the accrual override is in LUSID we need to address our second concern of notifying the aggregation to override the
accrual calculation with our value when required. We can achieve this via the use of a "Result Data Rule". These rules
rely on a pattern that when matched signals to the aggregation that the stored result data should be used over any
calculated value.

In [53]:
def create_accrual_result_data_rule(result_data_scope, accrual_result_id):
    # TODO exaplin this key in more detail not making much sense why it maps to accrual specifically
    accrual_key_rule = models.ResultDataKeyRule(
        resource_key="UnitResult/Isin/Yield",
        supplier="Client",
        data_scope=result_data_scope,
        document_code=accrual_result_id
    )

    return accrual_key_rule


accrual_key_rule = create_accrual_result_data_rule(result_data_scope, accrual_result_id)

Wit the rule create we need to instruct LUSID to apply this rule. To do so we simply add our rule to a an updated PricingContext definition
(which you recall tells LUSID what models to use for the aggregation).

In [146]:
def create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule):
    return models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True,
            produce_separate_result_for_linear_otc_legs=False,
            allow_partially_successful_evaluation=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ],
        result_data_rules=[accrual_key_rule]
    )

accrual_override_pricing_context = create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule)

As we've updated the pricing context we need to generate a new recipe for our portfolio valuation :

In [147]:

def create_bond_pricing_recipe_with_accrual_override(scope, recipe_name, market_context, pricing_context):

    discount_bond_pricing_config_recpie = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_name,
        description="Price bond using discounting model but override accurals",
        market=market_context,
        pricing=pricing_context
    )


    #upsert the recipe
    response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=discount_bond_pricing_config_recpie)
    )

    print(response)

    return discount_bond_pricing_config_recpie

accrual_override_recipe_name = "test-discounting-bond-acc-override"
accrual_override_config_recipe = create_bond_pricing_recipe_with_accrual_override(scope, accrual_override_recipe_name, market_context, accrual_override_pricing_context)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UECBBUE:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2020, 6, 29, 16, 0, 38, 782694, tzinfo=tzutc())}


Let's rerun the portfolio aggregation but this time with our accrual override and compare the results to running without an override

In [148]:
# run the aggregation with overrie (using accrual override recipe)
results_with_override = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, accrual_override_recipe_name, effective_at)
portfolio_pv_with_override = results_with_override.data[0]['Holding/default/PV']
#print(f"Bonds Holdings With Override (as at {effective_at}) : {portfolio_pv_with_override}")
accrued_interest_with_override = results_with_override.data[0]['Holding/default/Accrual']
#print(f"Accrued Interest With Override (as at {effective_at}) : {accrued_interest_with_override}")

# and now run without an override (using previously configured recipe).
results_no_override = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discounting_bond_recipe_name, effective_at)
portfolio_pv_no_override = results_no_override.data[0]['Holding/default/PV']
#print(f"Bonds Holdings No Override (as at {effective_at}) : {portfolio_pv_no_override}")
accrued_interest_no_override = results_no_override.data[0]['Holding/default/Accrual']
#print(f"Accrued Interest No Override (as at {effective_at}) : {accrued_interest_no_override}")


print(aggregation_result_to_dataframe([
    ['Portfolio PV (With Override)', effective_at, portfolio_pv_with_override],
    ['Portfolio Accrued Interest  (With Override)', effective_at, accrued_interest_with_override],
    ['Portfolio PV (No Override)', effective_at, portfolio_pv_no_override],
    ['Portfolio Accrued Interest (No Override)', effective_at, accrued_interest_no_override],
]))

Bonds Holdings With Override (as at 2020-06-23 00:00:00+00:00) : 108710319.79728633
Accrued Interest With Override (as at 2020-06-23 00:00:00+00:00) : 5183248.425
Bonds Holdings No Override (as at 2020-06-23 00:00:00+00:00) : 103998646.71475208
Accrued Interest No Override (as at 2020-06-23 00:00:00+00:00) : 471575.34246575343
                                          Name              Effective At  \
0                 Portfolio PV (With Override) 2020-06-23 00:00:00+00:00   
1  Portfolio Accrued Interest  (With Override) 2020-06-23 00:00:00+00:00   
2                   Portfolio PV (No Override) 2020-06-23 00:00:00+00:00   
3     Portfolio Accrued Interest (No Override) 2020-06-23 00:00:00+00:00   

          Value  
0  1.087103e+08  
1  5.183248e+06  
2  1.039986e+08  
3  4.715753e+05  


## Bond Price Override

Up until now we have priced our bond portfolio using LUSID's internal "Discounting" model. We'll now take a different approach and
upsert the market price into LUSID as quote. Continuing with our previous example where we processed a transaction at a clean price of 1.37,
now assume the market price of the bond has dropped to 1.35

Let's firstly upsert the latest market price as a quote into LUSID ensuring we map it to our specific Gilt via the instrument identifier:

In [155]:
def upsert_external_bond_price_as_quote(bond_price):
    spot_quote = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=gilt_2047_LUID,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                field='Mid'),
            effective_at=effective_at,
        ),
        metric_value=models.MetricValue(
            value=bond_price,
            unit='GBP'),
        #TODO describe lineage
        lineage='InternalSystem')

    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=market_data_scope,
        request_body={"1": spot_quote})

    print(response)

upsert_external_bond_price_as_quote(1.35)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UCPLPU8:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'1': {'as_at': datetime.datetime(2020, 6, 29, 16, 23, 3, 788406, tzinfo=tzutc()),
                  'cut_label': '',
                  'lineage': 'InternalSystem',
                  'metric_value': {'unit': 'GBP', 'value': 1.35},
                  'quote_id': {'effective_at': '2020-06-23T00:00:00.0000000+00:00',
                               'quote_series_id': {'field': 'Mid',
                                                   'instrument_id': 'LUID_I1RHNWII',
                                                   'instrument_id_type': 'LusidInstrumentId',
                                                   'price_source': None,
                   

As we now want to run a portfolio valuation using a different pricing model for Bonds we need a new recipe. Recall the two key components of the recipe
are the MaketContext and PricingContext. As we now have a new source of Market data, the quote we've inserted, we need to inform LUSID of where to locate it
during an aggregation. So in addition to the Rate rule we had in our precious MarketContext definition we add a new market rule for "Price":

In [152]:
def create_static_bond_pricing_market_context():
    return models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key='Equity.LusidInstrumentId.*',
                supplier=market_supplier,
                data_scope=market_data_scope,
                quote_type='Price',
                field='Mid'),
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope,
            manifest_level_of_detail="Full")
    )

    return market_context

static_bond_pricing_market_context = create_static_bond_pricing_market_context()

And we need to tell LUSID that we would like to used the quoted price of the bond and not use
the "Discounting" model. So we create a new PricingContext that uses a "SimpleStatic" model. To calculate PV of our bond
this model simply retrieves the market price and then adds the accrued interest (which itself could either be calculated or loaded
from the Structured Result Store as we covered earlier).


In [153]:
def create_static_bond_pricing_context():
    return models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True,
            produce_separate_result_for_linear_otc_legs=False,
            allow_partially_successful_evaluation=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                parameters="{}"
            )
        ]
    )

static_bond_pricing_context = create_static_bond_pricing_context()

With our updated Market and Pricing Contexts we can now generate a new recipe:

In [154]:
def create_static_bond_pricing_recipe(scope, recipe_name, market_context, pricing_context):

    static_bond_price_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_name,
        description="Price bond using prices from the quote store.",
        market=market_context,
        pricing=pricing_context
    )


    #upsert the recipe
    response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=static_bond_price_recipe)
    )

    print(response)

    return static_bond_price_recipe

static_bond_price_recipe_name = "test-static-bond"
create_static_bond_pricing_recipe(scope, static_bond_price_recipe_name, static_bond_pricing_market_context, static_bond_pricing_context)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UGBFQCN:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2020, 6, 29, 16, 22, 32, 68298, tzinfo=tzutc())}


{'aggregation': None,
 'code': 'test-static-bond',
 'description': 'Price bond using prices from the quote store.',
 'inherited_recipes': None,
 'market': {'market_rules': [{'as_at': None,
                              'data_scope': 'FinbourneMarketData',
                              'field': 'Mid',
                              'key': 'Equity.LusidInstrumentId.*',
                              'price_source': None,
                              'quote_interval': None,
                              'quote_type': 'Price',
                              'supplier': 'Lusid'},
                             {'as_at': None,
                              'data_scope': 'FinbourneMarketData',
                              'field': 'Mid',
                              'key': 'Rates.*.*',
                              'price_source': None,
                              'quote_interval': None,
                              'quote_type': 'Rate',
                              'supplier': 'Lusid'}],
 

Let's rerun our portfolio valuation using our static bond price model and compare the results to the valuations using our discounting model:

In [156]:
results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_recipe_name, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Holding/default/PV']
print(f"Bonds Holdings With Static Bond Price (as at {effective_at}) : {portfolio_pv_with_static_price}")
accrued_interest_with_static_price = results_using_static_price.data[0]['Holding/default/Accrual']
print(f"Accrued Interest With Static Bond Price (as at {effective_at}) : {accrued_interest_with_static_price}")

results_using_discounting = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discounting_bond_recipe_name, effective_at)
portfolio_pv_with_discounting = results_using_discounting.data[0]['Holding/default/PV']
print(f"Bonds Holdings Discounting Price (as at {effective_at}) : {portfolio_pv_with_discounting}")
accrued_interest_with_discounting = results_using_discounting.data[0]['Holding/default/Accrual']
print(f"Accrued Interest Discounting Price (as at {effective_at}) : {accrued_interest_with_discounting}")

print(aggregation_result_to_dataframe([
    ['Portfolio PV (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio PV (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
]))

Bonds Holdings With Static Bond Price (as at 2020-06-23 00:00:00+00:00) : 101721575.34246576
Accrued Interest With Static Bond Price (as at 2020-06-23 00:00:00+00:00) : 471575.34246575343
Bonds Holdings Discounting Price (as at 2020-06-23 00:00:00+00:00) : 103998646.71475208
Accrued Interest Discounting Price (as at 2020-06-23 00:00:00+00:00) : 471575.34246575343
                                            Name              Effective At  \
0                    Portfolio PV (Static Model) 2020-06-23 00:00:00+00:00   
1     Portfolio Accrued Interest  (Static Model) 2020-06-23 00:00:00+00:00   
2               Portfolio PV (Discounting Model) 2020-06-23 00:00:00+00:00   
3  Portfolio Accrued Interest (Discounting Mode) 2020-06-23 00:00:00+00:00   

          Value  
0  1.017216e+08  
1  4.715753e+05  
2  1.039986e+08  
3  4.715753e+05  


Update the market price of the bond to 1.39

In [157]:
upsert_external_bond_price_as_quote(1.39)

results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_recipe_name, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Holding/default/PV']
print(f"Bonds Holdings With Static Bond Price (as at {effective_at}) : {portfolio_pv_with_static_price}")
accrued_interest_with_static_price = results_using_static_price.data[0]['Holding/default/Accrual']
print(f"Accrued Interest With Static Bond Price (as at {effective_at}) : {accrued_interest_with_static_price}")

print(aggregation_result_to_dataframe([
    ['Portfolio PV (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio PV (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
]))

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UGBFQE6:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'1': {'as_at': datetime.datetime(2020, 6, 29, 16, 25, 46, 940956, tzinfo=tzutc()),
                  'cut_label': '',
                  'lineage': 'InternalSystem',
                  'metric_value': {'unit': 'GBP', 'value': 1.39},
                  'quote_id': {'effective_at': '2020-06-23T00:00:00.0000000+00:00',
                               'quote_series_id': {'field': 'Mid',
                                                   'instrument_id': 'LUID_I1RHNWII',
                                                   'instrument_id_type': 'LusidInstrumentId',
                                                   'price_source': None,
                  

Finally let's update the market price to equal the clean price as per the "Discounting" model. In this case we arrive at
the same portfolio PV using two different Bond Pricing models

In [161]:
# retrieve unit pv and accrued interest
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recpie, effective_at)
clean_as_per_disc_model = aggregation_result.data[0]['Holding/default/PV'] - aggregation_result.data[0]['Holding/default/Accrual']
upsert_external_bond_price_as_quote(clean_as_per_disc_model)

results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_recipe_name, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Holding/default/PV']
#print(f"Bonds Holdings With Static Bond Price (as at {effective_at}) : {portfolio_pv_with_static_price}")
accrued_interest_with_static_price = results_using_static_price.data[0]['Holding/default/Accrual']
#print(f"Accrued Interest With Static Bond Price (as at {effective_at}) : {accrued_interest_with_static_price}")


print(aggregation_result_to_dataframe([
    ['Portfolio PV (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio PV (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
]))

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0S5UECBCE5:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'1': {'as_at': datetime.datetime(2020, 6, 29, 16, 32, 21, 406857, tzinfo=tzutc()),
                  'cut_label': '',
                  'lineage': 'InternalSystem',
                  'metric_value': {'unit': 'GBP', 'value': 1.3803609516304842},
                  'quote_id': {'effective_at': '2020-06-23T00:00:00.0000000+00:00',
                               'quote_series_id': {'field': 'Mid',
                                                   'instrument_id': 'LUID_I1RHNWII',
                                                   'instrument_id_type': 'LusidInstrumentId',
                                                   'price_source': None,
    